In [1]:
from pyvi import ViTokenizer #For split vietnamese words
import pandas as pd #For reading xlsx file
from gensim.parsing.preprocessing import strip_non_alphanum, strip_multiple_whitespaces,preprocess_string, split_alphanum, strip_short, strip_numeric
import re
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score

In [2]:
def raw_text_preprocess(raw):    
    raw = ViTokenizer.tokenize(raw)
    raw = strip_short(raw, minsize=2)
    return raw

In [3]:
document = []
label = []
with open('data_email.txt', 'r', encoding = 'utf-8') as f:
    for line in f:
#         document_test.append(line[6: ])
        document.append(line[6: ])
        if ('non' in line[: 6]):
#             label.append(0)
            label.append(0)
        else:
#             label.append(1)
            label.append(1)

print(len(document))

407


In [4]:
def change(raw):
    raw_cop = raw
    token = raw.split(" ")
    links = ('http:', '.com', '.org', '.vn', '@', 'https:')
    for i, char in enumerate(token):
#         print(char)
        for link in links:
            if (link in char):
                token[i] = 'URL'
                break
    for i, tok in enumerate(token):
        if (tok[0].isdigit()):
            token[i] = 'number_change'
#     print(token)
    return token

In [5]:
document = [raw_text_preprocess(d) for d in document]
document_test = document[: 80]
label_test = label[: 80]
document = document[80 :]
label = label[80 :]


In [6]:
def dict_word(document):
    res = {}
    words = {}
    conten = 0
    for doc in document:
            token = change(doc)
            for tok in token:
                if tok in res.keys():
                    res[tok] = res[tok] + 1
                else:
                    res.update({tok : 1})
            for key, value in res.items():
                if value >= 3:
                    words.update({key : value})
            word = {}
            for tok in token:
                if tok in word.keys():
                    word[tok] = word[tok] + 1
                else:
                    word.update({tok : 1})
            conten = conten + len(word)
    words = sorted(words.keys())
    
    return (words, conten)

In [7]:
def read_data(document, label, words, conten):
    word_list = []
#     conten = 0
    for doc in document:
        token = change(doc)
        word = {}
        for tok in token:
            if tok in word.keys():
                word[tok] = word[tok] + 1
            else:
                word.update({tok : 1})
        word_list.append(word)
    j = 0
    dat = np.zeros((conten, 3), dtype = int)
    for i, word in enumerate(word_list):
        for key, value in word.items():
            if(key in words):
                dat[j, :] = np.array([int(i), int(words.index(key)), int(value)])
                j = j + 1
    data = coo_matrix((dat[:, 2], (dat[:, 0] , dat[:, 1] )),shape=(len(label), len(words)))
    return (data, label)

In [8]:
(words, conten) = dict_word(document)
(train_data, train_label) = read_data(document, label, words, conten)
(test_data, test_label) = read_data(document_test, label_test, words, conten)
clf = MultinomialNB()
clf.fit(train_data, train_label)
y_pred = clf.predict(test_data)
print('Training size =',train_data.shape[0],'. accuracy =', accuracy_score(y_pred,test_label)*100)

Training size = 327 . accuracy = 86.25
